In [8]:
import pandas as pd

file_path = "/Users/davidaslanyan/Documents/AUA I 2425Spring I Courses/Data Visualization/Homeworks_DV/crime_data.csv"
df = pd.read_csv(file_path)
print(df.head())
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Missing values per column:\n", missing_values)
df_cleaned = df.dropna(thresh=len(df) * 0.5, axis=1).copy()
df_cleaned.columns = df_cleaned.columns.str.strip().str.lower()
df_cleaned.loc[:, 'date occ'] = pd.to_datetime(df_cleaned['date occ'], errors='coerce')
df_cleaned.loc[:, 'year'] = df_cleaned['date occ'].dt.year
df_cleaned.loc[:, 'month'] = df_cleaned['date occ'].dt.month
df_cleaned.loc[:, 'day'] = df_cleaned['date occ'].dt.day
df_cleaned.loc[:, 'hour'] = df_cleaned['time occ'] // 100
df_2023 = df_cleaned[df_cleaned['year'] == 2023]
df_burglary = df_2023[df_2023['crm cd desc'] == 'BURGLARY']
crime_summary = df_cleaned.groupby('area name').agg(
    total_crimes=('crm cd desc', 'count'),
    average_victim_age=('vict age', 'mean')
).sort_values(by='total_crimes', ascending=False)
print(crime_summary.head())
top_crimes = df_cleaned['crm cd desc'].value_counts().head(3)
print("Top 3 Crimes:\n", top_crimes)
crime_by_hour = df_cleaned.groupby('hour').size()
print(crime_by_hour)
victim_summary = df_cleaned.groupby('vict sex').agg(
    total_crimes=('crm cd desc', 'count'),
    average_victim_age=('vict age', 'mean')
)
print(victim_summary)
def severity_score(row):
    if 'weapon used cd' in df_cleaned.columns and pd.notnull(row.get('weapon used cd')):
        return 5
    elif row['crm cd desc'] == 'BURGLARY':
        return 3
    else:
        return 1
df_cleaned['severity score'] = df_cleaned.apply(severity_score, axis=1)
severity_by_area = df_cleaned.groupby('area name')['severity score'].sum()
print(severity_by_area)


       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA  \
0  241711715  08/01/2024 12:00:00 AM  08/01/2024 12:00:00 AM      1319    17   
1  231014031  09/21/2023 12:00:00 AM  09/15/2023 12:00:00 AM      1930    10   
2  231010808  06/27/2023 12:00:00 AM  06/26/2023 12:00:00 AM      1230    10   
3  211410441  04/25/2021 12:00:00 AM  04/25/2021 12:00:00 AM      2330    14   
4  211114569  10/25/2021 12:00:00 AM  10/25/2021 12:00:00 AM      1455    11   

     AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0   Devonshire         1791         1     440   
1  West Valley         1011         2     354   
2  West Valley         1015         2     354   
3      Pacific         1488         2     626   
4    Northeast         1123         1     210   

                          Crm Cd Desc  ... Status  Status Desc Crm Cd 1  \
0  THEFT PLAIN - PETTY ($950 & UNDER)  ...     IC  Invest Cont    440.0   
1                   THEFT OF IDENTITY  ...     IC  Invest Cont    354.0   

/var/folders/cz/ysxc02l92kv04yfdk1rqmzx80000gn/T/ipykernel_86600/882316030.py:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_cleaned.loc[:, 'date occ'] = pd.to_datetime(df_cleaned['date occ'], errors='coerce')


             total_crimes  average_victim_age
area name                                    
Central              3517           28.481376
77th Street          3115           29.151525
Pacific              3004           28.978029
Southwest            2831           29.048040
Hollywood            2556           28.331377
Top 3 Crimes:
 VEHICLE - STOLEN            5733
BATTERY - SIMPLE ASSAULT    3715
THEFT OF IDENTITY           3169
Name: crm cd desc, dtype: int64
hour
0     2025
1     1490
2     1241
3     1063
4      975
5      848
6     1106
7     1302
8     1776
9     1841
10    2110
11    2114
12    3446
13    2246
14    2504
15    2667
16    2597
17    2914
18    3015
19    2762
20    2830
21    2553
22    2469
23    2106
dtype: int64
          total_crimes  average_victim_age
vict sex                                  
F                17922           38.164156
H                    3           36.333333
M                20076           37.165621
X                 4836            2